## Autoencoder to fill in gene data files

### 1. import libraries

In [15]:
import keras
import numpy as np
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, add
from keras.layers.core import Layer, Dense, Dropout, Activation, Flatten, Reshape
from keras import regularizers
from keras.regularizers import l2
from keras.layers.convolutional import Conv2D, MaxPooling2D, UpSampling2D, ZeroPadding2D
from keras.utils import np_utils

### 2. load data

In [26]:
from __future__ import print_function

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, RobustScaler


import tensorflow as tf

import time

now = time.time()
tag = str(now)

DEBUG = True   # always
LEARNING_RATE = 5e-6
TRAINING_ITERATIONS = 20000

# sizes of the hidden layers
NN_HL_ARCH = [100, 100, 100, 50]  
   
# probability of keeping a neuron = 1-prob(dropout)
DROPOUT = 1.0

# batch size 
BATCH_SIZE = 1 # there may be some errors if set to > 1. 

# set to 0 to train on all available data
# (currently validation is not used)
VALIDATION_SIZE = 0 #2000


if DEBUG:
    print('reading CSV input...')
df = pd.read_csv('/home/danni/Dropbox/CS/Code/gene_autoencoder/zeros_removed.csv')

print('df({0[0]},{0[1]})'.format(df.shape))
print (df.head())

if DEBUG:
    print('finished reading data...')

reading CSV input...
df(3143,184)
                 0        1    2         3        4         5         6    7  \
0  ENSG00000000419   0.0000  0.0   7.80771   0.0000   0.00000   0.00000  0.0   
1  ENSG00000002586  24.6584  0.0  55.47970  44.1613   0.00000   4.83525  0.0   
2  ENSG00000002834   0.0000  0.0   0.00000   0.0000   8.54906   9.77374  0.0   
3  ENSG00000003056  78.2393  0.0  72.54200   0.0000  70.38990   0.00000  0.0   
4  ENSG00000003402  12.9291  0.0   0.00000  12.8144   5.47288  22.74410  0.0   

         8        9       ...         174  175  176  177  178  179       180  \
0  204.495   0.0000       ...         0.0  0.0  0.0  0.0  0.0  0.0    0.0000   
1    0.000  95.1674       ...         0.0  0.0  0.0  0.0  0.0  0.0  133.0920   
2    0.000   0.0000       ...         0.0  0.0  0.0  0.0  0.0  0.0   17.1774   
3    0.000  55.4920       ...         0.0  0.0  0.0  0.0  0.0  0.0   74.4863   
4    0.000  25.3042       ...         0.0  0.0  0.0  0.0  0.0  0.0    1.7521   

    

### 3. Split dataset

80% training set and 20% testing set

In [47]:
if DEBUG:
    print('spliting into training and testing dataset...')
# df['split'] = np.random.randn(df.shape[0], 1)

# msk = np.random.rand(len(df)) <= 0.8

# train = df[msk]
# test = df[~msk]

validation_inputs = inputs[:VALIDATION_SIZE]
validation_labels = inputs[:VALIDATION_SIZE]

train_inputs = inputs[VALIDATION_SIZE:]
train_labels = inputs[VALIDATION_SIZE:]

input_size = len(inputs[0])
if DEBUG:
    print('finished splitting.')

spliting into training and testing dataset...
finished splitting.


In [49]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

if DEBUG:
    print('creating placeholders for input and output...')
    
# inputs
x = tf.placeholder('float', shape=[None,input_size])
# outputs = labels
y_ = tf.placeholder('float', shape=[None,input_size])

creating placeholders for input and output...


### 4. Prevent Overfitting

Dropout to prevent overfitting

In [50]:
keep_prob = tf.placeholder('float')
if DEBUG:
    print('creating W,b,h variables for various layers...')
    
W = []
b = []
h = []
h_tmp = []

if DEBUG:
    print('\tnow creating input layer...')
print('\t\t',input_size,',',NN_HL_ARCH[0])

W.append(weight_variable([input_size,NN_HL_ARCH[0]]))
b.append(bias_variable([NN_HL_ARCH[0]]))
h.append(tf.nn.relu(tf.matmul(x,W[0]) + b[0]))

for i in range(1,len(NN_HL_ARCH)):
    u = NN_HL_ARCH[i-1]
    v = NN_HL_ARCH[i]
    if DEBUG:
        print('\tnow creating layer:', i)
        print('\t\t',u,',',v)

    W.append(weight_variable([u,v]))
    b.append(bias_variable([v]))
    if DEBUG:
        pass
        #print('h', tf.shape(h[i-1]), ', W:', tf.shape(W[i]), ', b:', tf.shape(b[i]))

    h_tmp.append(tf.nn.relu(tf.matmul(h[i-1],W[i]) + b[i]))
    # dropout
    h.append(tf.nn.dropout(h_tmp[-1], keep_prob))

if DEBUG:
    print('\tnow creating output layer...')
    print('\t\t',NN_HL_ARCH[-1],',',input_size)

W.append(weight_variable([NN_HL_ARCH[-1],input_size]))
b.append(bias_variable([input_size]))
if DEBUG:
    print('\tsetting up output vector...')
y = tf.nn.relu(tf.matmul(h[-1],W[-1]) + b[-1])

creating W,b,h variables for various layers...
	now creating input layer...
		 183 , 100
	now creating layer: 1
		 100 , 100
	now creating layer: 2
		 100 , 100
	now creating layer: 3
		 100 , 50
	now creating output layer...
		 50 , 183
	setting up output vector...


### 5. Optimization

cost function

use gradiant based optimization algorithm: ADAM optimizaer

In [51]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y))
if DEBUG:
    print('defining objective function...')
# but we shall use rmse
rmse = tf.sqrt(tf.reduce_mean(tf.pow(y-y_, 2)))

if DEBUG:
    print('defining optimization step...')
# optimisation function
train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(rmse)

# evaluation
if DEBUG:
    print('defining optimization step...')

# CHECK
error_sq_vector = tf.pow(y - y_,2)

# CHECK
accuracy = tf.sqrt(tf.reduce_mean(error_sq_vector))
predict = tf.identity(y)


defining objective function...
defining optimization step...
defining optimization step...


### 6. Training

Use small batches first to avoid huge cost

In [52]:
epochs_completed = 0
index_in_epoch = 0
num_examples = train_inputs.shape[0]

# serve data by batches
def next_batch(batch_size):
    
    global train_inputs
    global train_labels
    global index_in_epoch
    global epochs_completed
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    # when all trainig data have been already used, it is reorder randomly    
    if index_in_epoch > num_examples:
        # finished epoch
        epochs_completed += 1
        # shuffle the data
        perm = np.arange(num_examples)
        np.random.shuffle(perm)
        train_inputs = train_inputs[perm]
        train_labels = train_labels[perm]
        # start next epoch
        start = 0
        index_in_epoch = batch_size
        assert batch_size <= num_examples
    end = index_in_epoch
    return train_inputs[start:end], train_labels[start:end]

In [53]:
init = tf.initialize_all_variables()
sess = tf.InteractiveSession()

sess.run(init)

In [57]:
train_accuracies = []
validation_accuracies = []
x_range = []

display_step=1

for i in range(TRAINING_ITERATIONS):

    #get new batch
    batch_xs, batch_ys = next_batch(BATCH_SIZE)        

    # check progress on every 1st,2nd,...,10th,20th,...,100th... step
    if i%display_step == 0 or (i+1) == TRAINING_ITERATIONS:
        
        train_accuracy = accuracy.eval(feed_dict={x:batch_xs, 
                                                  y_: batch_ys, 
                                                  keep_prob: 1.0})       
        if(VALIDATION_SIZE):
            validation_accuracy = accuracy.eval(feed_dict={ x: validation_inputs[0:BATCH_SIZE], 
                                                            y_: validation_labels[0:BATCH_SIZE], 
                                                            keep_prob: 1.0})                                  
            print('training_accuracy / validation_accuracy => %.2f / %.2f for step %d'%(train_accuracy, validation_accuracy, i))
            
            validation_accuracies.append(validation_accuracy)
            
        else:
             print('training_accuracy => %.4f for step %d'%(train_accuracy, i))
        train_accuracies.append(train_accuracy)
        x_range.append(i)
        
        # increase display_step
        if i%(display_step*10) == 0 and i:
            display_step *= 10
    # train on batch
sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys, keep_prob: DROPOUT})

training_accuracy => 45.7149 for step 0
training_accuracy => 87.1401 for step 1
training_accuracy => 21.0431 for step 2
training_accuracy => 55.4242 for step 3
training_accuracy => 29.0490 for step 4
training_accuracy => 30.0902 for step 5
training_accuracy => 69.0275 for step 6
training_accuracy => 638.4905 for step 7
training_accuracy => 20.6792 for step 8
training_accuracy => 59.5783 for step 9
training_accuracy => 111.9487 for step 10
training_accuracy => 13.8301 for step 20
training_accuracy => 39.0512 for step 30
training_accuracy => 21891.3125 for step 40
training_accuracy => 518.8159 for step 50
training_accuracy => 52.6602 for step 60
training_accuracy => 30.6682 for step 70
training_accuracy => 23.8834 for step 80
training_accuracy => 155.1471 for step 90
training_accuracy => 36.1892 for step 100
training_accuracy => 49.4693 for step 200
training_accuracy => 151.5613 for step 300
training_accuracy => 74.5648 for step 400
training_accuracy => 43.7708 for step 500
training_accu

In [60]:
if(VALIDATION_SIZE):
    validation_accuracy = accuracy.eval(feed_dict={x: validation_inputs, 
                                                   y_: validation_labels, 
                                                   keep_prob: 1.0})
    print('validation_accuracy => %.4f'%validation_accuracy)
    plt.plot(x_range, train_accuracies,'-b', label='Training')
    plt.plot(x_range, validation_accuracies,'-g', label='Validation')
    plt.legend(loc='lower right', frameon=False)
    plt.ylim(ymax = 1.1, ymin = 0.7)
    plt.ylabel('accuracy')
    plt.xlabel('step')
    plt.show()
    print('finished.')